In [3]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

import re
import pandas as pd
from tqdm import tqdm
from loguru import logger
from pymongo import MongoClient
import requests 
import time 

# GitHub Personal Access Token (optional, but recommended for higher rate limits)
GITHUB_TOKEN = os.getenv("GITHUB_TOKEN")
if not GITHUB_TOKEN:
    logger.warning("GitHub token not found. Rate limits may apply.")

BATCH_SIZE = 500 # Number of records to process before writing to CSV

# Logger config
logger.add("logs/pr-issues-linking-commite.log")


1

In [4]:
# Mongo connection
client = MongoClient("mongodb://localhost:27017/")
db = client.github_data
pull_request_col = db.pull_requests

logger.info("🔌 MongoDB connected.")

2025-05-27 10:08:21.458 | INFO     | __main__:<module>:6 - 🔌 MongoDB connected.


In [6]:
qurey = { "pull_request.state": "closed","timeline.event":"referenced" }
cursor = pull_request_col.find(qurey)
logger.info("🔍 MongoDB query executed.")

2025-05-27 10:08:26.011 | INFO     | __main__:<module>:3 - 🔍 MongoDB query executed.


In [4]:
print(pull_request_col.count_documents(qurey))

998


In [8]:
def get_commit_data(url):
    """
    Fetch commit data for a given issue number from GitHub API.
    """
    headers = {
        "Authorization": f"token {GITHUB_TOKEN}" if GITHUB_TOKEN else None
    }
    
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        html_url = data.get("html_url")
        message = data.get("commit").get("message")
        if not message:
            logger.error(f"Missing commit message in response: {data}")   
        logger.success(f"✅ Successfully fetched commit data from {url}")
        return html_url, message
    else:
        logger.error(f"Error fetching commit data from {url}: {response.status_code} - {response.text}")
        return None, None

In [6]:
results_commite_id = []

csv_path = "data/linked_pr_to_issues_commite_id.csv"

headers_pull_request_commite_id = [
    "pull_request_number", "pull_request_link", "pull_request_title", "pull_request_body",
    "linked_commit_id", "linked_commit_url", "linked_commit_message",
]

pd.DataFrame(columns=headers_pull_request_commite_id).to_csv(csv_path, index=False)
logger.info(f"Initialized CSV file at {csv_path}.")

processed_pull_request_count = 0
toal_pull_request_count = pull_request_col.count_documents(qurey)
logger.info(f"Total Pull_request to process: {toal_pull_request_count}.")
cursor = pull_request_col.find(qurey)

logger.info("starting to process pull_requests...")
for record in tqdm(cursor, total=toal_pull_request_count,desc="process Pull_request  (Pass 1)"):
    pull_request_data = record["pull_request"]
    timeline_data = record["timeline"]

    pull_request_number = pull_request_data["number"]
    pull_request_link = pull_request_data.get("html_url", None)
    pull_request_title = pull_request_data.get("title", None)
    pull_request_body = pull_request_data.get("body", None)

    for event in record["timeline"]:
        if event["event"] == "referenced":
            linked_commit_id = event.get("commit_id", None)
            linked_commit_url = event.get("commit_url", None)
            linked_commit_html_url,linked_commit_message = get_commit_data(linked_commit_url)
            if linked_commit_id:
                results_commite_id.append({
                    "pull_request_number": pull_request_number,
                    "pull_request_link": pull_request_link,
                    "pull_request_title": pull_request_title,
                    "pull_request_body": pull_request_body,
                    "linked_commit_id": linked_commit_id,
                    "linked_commit_url": linked_commit_html_url,
                    "linked_commit_message": linked_commit_message,
                })
    
    processed_pull_request_count += 1

    if processed_pull_request_count % BATCH_SIZE == 0 or processed_pull_request_count == toal_pull_request_count:
        if results_commite_id:
            df_batch = pd.DataFrame(results_commite_id)
            df_batch.to_csv(csv_path, mode='a', header=False, index=False)
            logger.info(f"Appended {len(results_commite_id)} records to {csv_path}.")
            results_commite_id = []  # Clear the list for the next batch

# Final check to ensure any remaining records are written
if results_commite_id:
    df_batch = pd.DataFrame(results_commite_id)
    df_batch.to_csv(csv_path, mode='a', header=False, index=False)
    logger.info(f"Appended remaining {len(results_commite_id)} records to {csv_path}.")

logger.info("✅ Processing complete.")

2025-05-27 08:13:17.153 | INFO     | __main__:<module>:11 - Initialized CSV file at data/linked_pr_to_issues_commite_id.csv.
2025-05-27 08:13:17.691 | INFO     | __main__:<module>:15 - Total Pull_request to process: 998.
2025-05-27 08:13:17.700 | INFO     | __main__:<module>:18 - starting to process pull_requests...
process Pull_request  (Pass 1):   1%|          | 9/998 [00:06<11:36,  1.42it/s]2025-05-27 08:13:24.756 | SUCCESS  | __main__:get_commit_data:17 - ✅ Successfully fetched commit data from https://api.github.com/repos/lochana-chathura/ballerina-lang/commits/c14cd0fd297af2515b980c68a2d7353aec989d06
2025-05-27 08:13:25.367 | SUCCESS  | __main__:get_commit_data:17 - ✅ Successfully fetched commit data from https://api.github.com/repos/lochana-chathura/ballerina-lang/commits/122e109de00a40ed24eb1886e996cfcf284720ad
process Pull_request  (Pass 1):   1%|          | 11/998 [00:08<13:01,  1.26it/s]2025-05-27 08:13:26.599 | SUCCESS  | __main__:get_commit_data:17 - ✅ Successfully fetched

In [ ]:
# Load the CSV file into a DataFrame
csv_path = "data/linked_pr_to_issues_commite_id.csv"
df = pd.read_csv(csv_path)

# Function to check if the commit message contains the pull_request number
def contains_pull_request_number(row):
    # Ensure pull_request_number is a string for searching
    pull_request_num_str = str(row['pull_request_number'])
    # Handle cases where linked_commit_message might be NaN or not a string
    commit_message = str(row['linked_commit_message'])
    if pd.isna(commit_message):
        return False
    # Check for the pull_request number in various common formats (e.g., #123, Fixes 123, pull_request 123)
    # Using regex for more flexible matching, ensuring it's the whole number
    patterns = [
        rf'#{pull_request_num_str}\b', 
        rf'\bpull_request\s+#?{pull_request_num_str}\b',
        rf'\bfix(?:es|ed)?\s+#?{pull_request_num_str}\b',
        rf'\bclose(?:s|d)?\s+#?{pull_request_num_str}\b',
        rf'\bresolve(?:s|d)?\s+#?{pull_request_num_str}\b',
        rf'\bgh-{pull_request_num_str}\b' # Common GitHub reference
    ]
    for pattern in patterns:
        if re.search(pattern, commit_message, re.IGNORECASE):
            return True
    return False

# Apply the function to create the new column
df['message_contains_pull_request_number'] = df.apply(contains_pull_request_number, axis=1)

print(f"\nNumber of messages containing relevant pull_request number: {df['message_contains_pull_request_number'].sum()}")
df.to_csv("message_contains_pull_request_number.csv", index=False)


Number of messages containing relevant pull_request number: 2268


In [8]:
df.head(100)

,pull_request_number,pull_request_link,pull_request_title,pull_request_body,linked_commit_id,linked_commit_url,linked_commit_message,message_contains_pull_request_number
0,44047,https://github.com/ballerina-platform/ballerin...,[master][LS] Fix IndexOutOfBoundException in S...,## Purpose\r\n$subject. \r\n\r\nFixes https://...,4e800422f10a07515fbf1eaedd9005411e0d2a0f,https://github.com/ballerina-platform/ballerin...,Merge pull request #44057 from ballerina-platf...,True
1,44046,https://github.com/ballerina-platform/ballerin...,[Master] Introduce a new api to get documentat...,## Purpose\r\n$subject\r\n\r\nThe changes has ...,4e800422f10a07515fbf1eaedd9005411e0d2a0f,https://github.com/ballerina-platform/ballerin...,Merge pull request #44057 from ballerina-platf...,True
2,43876,https://github.com/ballerina-platform/ballerin...,[2201.12.x] Add raw template type to Semantic ...,(cherry picked from commit 47fc7b5fff8ea895d37...,6d426c39abebe37af8892710cdf2d9b1235a688a,https://github.com/dulajdilshan/ballerina-lang...,Merge pull request #4 from ballerina-platform/...,True
3,43701,https://github.com/ballerina-platform/ballerin...,Improve performance of array add operation an...,## Purpose\r\nFixes https://github.com/balleri...,ce793f94a38df5a844d84519473e8b04317e60bd,https://github.com/chiranSachintha/ballerina-l...,"Revert ""Merge pull request #43701 from warunal...",True
4,43689,https://github.com/ballerina-platform/ballerin...,Optimize BDD operations,## Purpose\r\nAvoid repeated BDD operation usi...,9b1b66778270ad96f070a40ecd13f1664b590b3f,https://github.com/chiranSachintha/ballerina-l...,"Revert ""Merge pull request #43689 from heshanp...",True
...,...,...,...,...,...,...,...,...
95,30204,https://github.com/ballerina-platform/ballerin...,Handle error for mocking imported functions,## Purpose\r\nAdds an extra check to see if th...,783d35ad1a3650be519024e441b052e11851817e,https://github.com/Ibaqu/ballerina-lang/commit...,"Revert ""Merge pull request #30204 from Ibaqu/m...",True
96,30204,https://github.com/ballerina-platform/ballerin...,Handle error for mocking imported functions,## Purpose\r\nAdds an extra check to see if th...,2331ee332dd2e43f05a94f2028c9e067047079a1,https://github.com/IMS94/ballerina-lang/commit...,"Revert ""Merge pull request #30204 from Ibaqu/m...",True
97,29803,https://github.com/ballerina-platform/ballerin...,Update hashcode for `BFunction` type,## Purpose\r\n> $title.\r\n\r\nFixes #29795 \r...,9632ed9ad9af98fc1596d75a05a2e19cae8d98f2,https://github.com/ballerina-platform/ballerin...,Address review suggestions for #29803,True
98,29803,https://github.com/ballerina-platform/ballerin...,Update hashcode for `BFunction` type,## Purpose\r\n> $title.\r\n\r\nFixes #29795 \r...,370335339026651ad227f84ca26cb8568749670c,https://github.com/ballerina-platform/ballerin...,Merge pull request #29934 from ballerina-platf...,True


In [20]:
from typing import Optional, Tuple

def parse_commit_url(commit_url: Optional[str]) -> Tuple[Optional[str], Optional[str]]:
    """
    Parses the owner and repository name from a GitHub commit URL.

    Args:
        commit_url: The GitHub commit URL (either HTML or API endpoint).
                    Examples:
                    - https://github.com/owner/repo/commit/sha
                    - https://api.github.com/repos/owner/repo/commits/sha

    Returns:
        A tuple (owner, repo) if parsing is successful, otherwise (None, None).
    """
    # Handle None, empty, or non-string inputs
    if not commit_url or not isinstance(commit_url, str):
        if commit_url is not None:  # Only log if it's not None
            logger.warning(f"Invalid commit_url (type: {type(commit_url)}): '{commit_url}'")
        return None, None

    commit_url_str = commit_url.strip()
    if not commit_url_str:
        logger.warning("Empty commit_url after stripping whitespace")
        return None, None

    # Combined regex pattern to match both HTML and API URLs
    # Group 1: owner, Group 2: repo
    pattern = r"https?://(?:(?:www\.)?github\.com/([^/]+)/([^/]+)/commit/|api\.github\.com/repos/([^/]+)/([^/]+)/commits/)"
    
    match = re.search(pattern, commit_url_str)
    if match:
        # HTML URL match (groups 1, 2) or API URL match (groups 3, 4)
        owner = match.group(1) or match.group(3)
        repo = match.group(2) or match.group(4)
        
        # Truncate URL for logging
        truncated_url = commit_url_str[:100] + ('...' if len(commit_url_str) > 100 else '')
        logger.debug(f"Parsed URL '{truncated_url}': owner='{owner}', repo='{repo}'")
        return owner, repo
    
    # Log failure with truncated URL
    truncated_url = commit_url_str[:100] + ('...' if len(commit_url_str) > 100 else '')
    logger.warning(f"Could not parse owner/repo from URL: {truncated_url}")
    return None, None

In [21]:
print(parse_commit_url("https://github.com/IMS94/ballerina-lang/commit/e1a58682a8fd8bc46df0350946d8d0e798caf0ce"))

2025-05-27 10:54:11.840 | DEBUG    | __main__:parse_commit_url:39 - Parsed URL 'https://github.com/IMS94/ballerina-lang/commit/e1a58682a8fd8bc46df0350946d8d0e798caf0ce': owner='IMS94', repo='ballerina-lang'


('IMS94', 'ballerina-lang')


In [19]:
# Function to fetch Issue data for a commit using the search API
def get_issue_data_for_commit(owner, repo, commit_sha):
    if not all([owner, repo, commit_sha]):
        logger.error(f"Missing owner, repo, or commit_sha for API call. Owner: {owner}, Repo: {repo}, SHA: {commit_sha}")
        return None, None, None, None
    # Using the search API to find issues associated with a commit SHA
    # The query targets issues in the specific repo that mention the commit SHA and are issues
    api_url = f"https://api.github.com/search/issues?q=repo:{owner}/{repo}+sha:{commit_sha}+is:issue"
    headers = {
        "Authorization": f"token {GITHUB_TOKEN}" if GITHUB_TOKEN else None,
        "Accept": "application/vnd.github.v3+json" 
    }
    
    try:
        response = requests.get(api_url, headers=headers, timeout=30) # Added timeout
        
        # Handle rate limits proactively
        if 'X-RateLimit-Remaining' in response.headers and int(response.headers['X-RateLimit-Remaining']) < 10: # Search API has lower rate limits
            reset_time = int(response.headers.get('X-RateLimit-Reset', time.time() + 60))
            sleep_duration = max(0, reset_time - time.time()) + 10 # Increased buffer
            logger.warning(f"Search API rate limit low ({response.headers['X-RateLimit-Remaining']}). Sleeping for {sleep_duration:.0f} seconds.")
            time.sleep(sleep_duration)
        if response.status_code == 200:
            search_results = response.json()
            if search_results.get("items") and len(search_results["items"]) > 0:
                # Assuming the first issue found is the most relevant one
                issue_data = search_results["items"][0]
                logger.success(f"✅ Issue data for commit {commit_sha[:7]} in {owner}/{repo}: Issue #{issue_data.get('number')}")
                return issue_data.get("number"), issue_data.get("html_url"), issue_data.get("title"), issue_data.get("body")
            else:
                logger.info(f"ℹ️ No issues found associated with commit {commit_sha[:7]} in {owner}/{repo} via search API.")
                return None, None, None, None
        elif response.status_code == 403: # Handles primary and secondary rate limits for search
            reset_time_str = response.headers.get('X-RateLimit-Reset')
            retry_after_str = response.headers.get('Retry-After')
            
            sleep_duration = 60 # Default sleep
            if retry_after_str:
                sleep_duration = int(retry_after_str) + 5 # Use Retry-After if available
            elif reset_time_str:
                reset_time = int(reset_time_str)
                sleep_duration = max(0, reset_time - time.time()) + 15
            
            logger.error(f"Rate limit exceeded or access forbidden for {api_url} (Status 403). Response: {response.text[:200]}. Sleeping for {sleep_duration:.0f} seconds.")
            time.sleep(sleep_duration)
            return None, None, None, None # Indicate failure for this attempt
        elif response.status_code == 422: # Unprocessable Entity - often due to query validation
            logger.error(f"Validation failed for API call {api_url} (Status 422): {response.text[:200]}. This might be due to the commit SHA not being indexed or an issue with the search query.")
            return None, None, None, None
        else:
            logger.error(f"Error fetching issue data for commit {commit_sha[:7]} from {api_url}: {response.status_code} - {response.text[:200]}")
            return None, None, None, None
    except requests.exceptions.Timeout:
        logger.error(f"Request timed out for {api_url}")
        return None, None, None, None
    except requests.exceptions.RequestException as e:
        logger.error(f"Request failed for {api_url}: {e}")
        return None, None, None, None

# Initialize lists for new issue data
issue_numbers = []
issue_links = []
issue_titles = []
issue_bodies = []

logger.info("🚀 Starting to fetch issue data for commits...")

# Iterate over the DataFrame with a progress bar
for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Fetching issue data"):
    commit_url = row['linked_commit_url']
    commit_sha = row['linked_commit_id']

    if pd.isna(commit_url) or pd.isna(commit_sha):
        logger.warning(f"Skipping row {index} due to missing commit_url ('{commit_url}') or commit_sha ('{commit_sha}').")
        issue_numbers.append(None)
        issue_links.append(None)
        issue_titles.append(None)
        issue_bodies.append(None)
        continue

    owner, repo = parse_commit_url(str(commit_url)) # Ensure commit_url is string

    if owner and repo:
        issue_number_val, issue_link_val, issue_title_val, issue_body_val = get_issue_data_for_commit(owner, repo, str(commit_sha))
        issue_numbers.append(issue_number_val)
        issue_links.append(issue_link_val)
        issue_titles.append(issue_title_val)
        issue_bodies.append(issue_body_val)
    else:
        # parse_commit_url would have logged the warning
        issue_numbers.append(None)
        issue_links.append(None)
        issue_titles.append(None)
        issue_bodies.append(None)
    
    # A small, fixed delay can be added here if GITHUB_TOKEN is not available or if issues persist
    # However, the dynamic sleep in get_issue_data_for_commit based on headers is generally better.
    # if not GITHUB_TOKEN: time.sleep(1) # Example: 1 sec delay if no token

# Add new columns to the DataFrame
df['issue_number'] = issue_numbers
df['issue_link'] = issue_links
df['issue_title'] = issue_titles
df['issue_body'] = issue_bodies

logger.info("✅ Issue data fetching complete. New columns added to DataFrame.")

# Display some info about the new columns
print("\\nDataFrame with new issue columns (showing PR-to-Issue link, first 5 rows):")
print(df[['pull_request_number', 'linked_commit_id', 'issue_number', 'issue_link', 'issue_title']].head())

issue_found_count = df['issue_number'].notna().sum()
print(f"\\nNumber of commits for which issue data was successfully found: {issue_found_count} out of {df.shape[0]}")

# Optionally, save the updated DataFrame to a new CSV file
# output_csv_path_with_issue_data = "data/linked_issues_commit_issue_data.csv"
# df.to_csv(output_csv_path_with_issue_data, index=False)
# logger.info(f"💾 Updated DataFrame potentially saved to {output_csv_path_with_issue_data}")


2025-05-27 10:45:20.657 | INFO     | __main__:<module>:66 - 🚀 Starting to fetch issue data for commits...
Fetching issue data:   0%|          | 0/2308 [00:00<?, ?it/s]2025-05-27 10:45:20.657 | DEBUG    | __main__:parse_commit_url:39 - Parsed URL 'https://github.com/ballerina-platform/ballerina-lang/commit/4e800422f10a07515fbf1eaedd9005411e0d2a0f': owner='ballerina-platform', repo='ballerina-lang'


2025-05-27 10:45:21.106 | INFO     | __main__:get_issue_data_for_commit:31 - ℹ️ No issues found associated with commit 4e80042 in ballerina-platform/ballerina-lang via search API.
Fetching issue data:   0%|          | 1/2308 [00:00<17:16,  2.23it/s]2025-05-27 10:45:21.106 | DEBUG    | __main__:parse_commit_url:39 - Parsed URL 'https://github.com/ballerina-platform/ballerina-lang/commit/4e800422f10a07515fbf1eaedd9005411e0d2a0f': owner='ballerina-platform', repo='ballerina-lang'
2025-05-27 10:45:21.572 | INFO     | __main__:get_issue_data_for_commit:31 - ℹ️ No issues found associated with commit 4e80042 in ballerina-platform/ballerina-lang via search API.
Fetching issue data:   0%|          | 2/2308 [00:00<17:41,  2.17it/s]2025-05-27 10:45:21.574 | DEBUG    | __main__:parse_commit_url:39 - Parsed URL 'https://github.com/dulajdilshan/ballerina-lang/commit/6d426c39abebe37af8892710cdf2d9b1235a688a': owner='dulajdilshan', repo='ballerina-lang'
2025-05-27 10:45:21.572 | INFO     | __main__:ge

KeyboardInterrupt: 

In [ ]:
output_csv_path_with_pr = "data/linked_pr_commit_id_pr_data.csv"
df.to_csv(output_csv_path_with_pr, index=False)
logger.info(f"💾 Updated DataFrame potentially saved to {output_csv_path_with_pr}")